In [2]:
from dotenv import load_dotenv
import pandas as pd
from datetime import date, datetime
from src.model.repositorios.xls.movimentacao_estoque import MovimentacaoDeEstoqueXls

In [ ]:
load_dotenv()

In [4]:
mov_estoque = MovimentacaoDeEstoqueXls()

In [5]:
dt_inicial = datetime(2024, 1, 1)
dt_final = datetime(2024, 8, 30)

mov = mov_estoque.buscar_movimentacao(dt_inicial=dt_inicial, dt_final=dt_final)

### Carregando o Dataset

Vendas = 'NVD'
Pedidos de Venda = 'PVD'
Pedidos de Compra 'PCP'
Compras 'C' = 'NCP'
Devoluções de compra = 'DCP'
Devoluções de venda = 'DVD'

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df['descricao'] = df['descricao'].str.capitalize()
df['marca'] = df['marca'].str.capitalize()

# df['tipo_movimento'] = df['tipo_movimento'].str.capitalize()
df['segmento'] = df['segmento'].str.capitalize()
df['cidade'] = df['cidade'].str.capitalize()
df['secao'] = df['secao'].str.capitalize()
df['grupo'] = df['grupo'].str.capitalize()

df['data_movimento'] = pd.to_datetime(df['data_movimento'], format='%d/%m/%Y %H:%M:%S')
df['valor_total'] = df['qtd_negociada'] * (df['vlr_unitario'] - df['vlr_desconto'])

In [ ]:
df.sort_values('data_movimento', ascending=False)
df.head()

In [4]:
columns_names: set = {
        'nro_documento',
        'tipo_movimento',
        'cod_parceiro',
        'segmento',
        'data_movimento',
        'cidade',
        'cod_produto',
        'descricao',
        'marca',
        'secao',
        'grupo',
        'qtd_negociada',
        'vlr_unitario',
        'vlr_desconto'
    }

In [ ]:
{item: item in df.columns for item in columns_names}

In [ ]:
df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,14), datetime(2024,9,16), inclusive='neither'))].sort_values('data_movimento', ascending=False)

In [ ]:
#df_marca = df[df['tipo_movimento'] == 'V'].groupby(['marca'])[['qtd_negociada', 'vlr_unitario']].sum().reset_index()

df_marca = df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,15), datetime(2024,9,17)))]\
    .groupby(['marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'vlr_unitario': 'sum'})\
    .sort_values('vlr_unitario', ascending=False)\
    .reset_index()

df_marca['vlr_unitario'] = df_marca['vlr_unitario'].apply(lambda x: round(x, 2))
df_marca

### Calendario

In [15]:
from workalendar.america import Brazil
cal = Brazil()


dt_inicial = date(2024,1,1)
dt_final = date(2024,6,30)
cal.get_working_days_delta(dt_inicial, dt_final)

cal.holidays(2024).__contains__

<method-wrapper '__contains__' of list object at 0x0000020005CCDC80>

In [16]:
dates = pd.date_range(start='2024-01-01', end='2024-06-30')
calendar = pd.DataFrame(None, dates.rename('date'), 'dia mes ano semana dia_semana dia_util'.split())
calendar.dropna()

calendar = calendar.assign(
    dia=dates.day,
    mes=dates.month,
    ano=dates.year,
    dia_semana=dates.day_of_week,
    dia_util=calendar.index.map(cal.is_working_day)
)
calendar.loc[calendar['dia_semana']==5, ['dia_util']] = True

iso = dates.isocalendar()
calendar = calendar.assign(semana=iso.week)

calendar.index.dtype

dtype('<M8[ns]')

In [ ]:
dta_inicial = pd.Timestamp(date(2024,8,1))
dta_final = pd.Timestamp(date(2024,8,15))
len(calendar[calendar['dia_util'] == True].loc[dta_inicial:dta_final])

### Produtos

In [22]:
dta_inicial = pd.Timestamp(date(2024,1,1))
dta_final = pd.Timestamp(date(2024,6,30))
dias_uteis = len(calendar[calendar['dia_util'] == True].loc[dta_inicial:dta_final])

df_produtos = df[(df['tipo_movimento'] == 'NVD') & (df['data_movimento'].between(datetime(2024,1,1), datetime(2024,6,30)))]\
    .groupby(['segmento', 'cod_produto', 'descricao', 'marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'valor_total': 'sum'})\
    .sort_values('valor_total', ascending=False)\
    .reset_index()

df_produtos['perc_vendas'] = df_produtos[['valor_total']] / df_produtos[['valor_total']].sum()

total = round(df_produtos[['valor_total']].sum(), 2)
df_produtos['valor_total_acu'] = round(df_produtos['valor_total'].cumsum(), 2)

df_produtos['perc_vendas_acu'] = df_produtos[['perc_vendas']].cumsum()

df_produtos['curva_abc_valor'] = ''
df_produtos['curva_abc_vendas'] = ''
#df_produtos[df_produtos['perc_vendas_acu'] <= 0.20]['curva_abc'] = 'A'

df_produtos.loc[df_produtos['perc_vendas_acu'] <= 0.20, ['curva_abc_valor']] = 'A'
df_produtos.loc[df_produtos['perc_vendas_acu'].between(0.20, 0.50), ['curva_abc_valor']] = 'B'
df_produtos.loc[df_produtos['perc_vendas_acu'] > 0.50, ['curva_abc_valor']] = 'C'

df_produtos['qtd_media_diaria'] = round(df_produtos['qtd_negociada'] / dias_uteis, 0)
#df_produtos[df_produtos['marca'].str.lower() == 'krona']
df_produtos[df_produtos['cod_produto'] == 31615]

,segmento,cod_produto,descricao,marca,nro_documento,qtd_negociada,valor_total,perc_vendas,valor_total_acu,perc_vendas_acu,curva_abc_valor,curva_abc_vendas,qtd_media_diaria
24097,Materiais para construção,31615,Luva p/eletr. 1/2 (5),Krona,10,325.0,266.75,3.290666e-06,77762583.64,0.959290,C,,2.0
34464,Material hidráulico / elétrico,31615,Luva p/eletr. 1/2 (5),Krona,5,150.0,126.70,1.562989e-06,79696763.27,0.983151,C,,1.0
56227,Produtos agropecuários,31615,Luva p/eletr. 1/2 (5),Krona,2,15.0,12.50,1.542018e-07,81052075.34,0.999870,C,,0.0
57341,Auto-peças e acessórios,31615,Luva p/eletr. 1/2 (5),Krona,1,5.0,4.30,5.304541e-08,81062161.83,0.999994,C,,0.0
57342,Ferragens e ferramentas,31615,Luva p/eletr. 1/2 (5),Krona,1,5.0,4.30,5.304541e-08,81062166.13,0.999994,C,,0.0
